In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
witness_path = os.path.join('prover/witness.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [5]:
data_path = os.path.join('data.json')

In [6]:
scales = [2]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

## Step 2
- User defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `computation_to_model(computation)` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [7]:
from zkstats.computation import State, computation_to_model


def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

_, prover_model = computation_to_model(computation)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_37051/4262130413.py:7: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_37051/4262130413.py:7: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:75: TracerWarning: torch.tensor results are registered as constants in

==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-194,338],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":2624,"total_assignments":370,"total_const_size":74,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2,2],"module_sizes":{"kzg":[],"poseidon":[2624,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(prover_model_path, prover_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 0.8651909828186035 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.75
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[3403910923874899465, 10189271995497415241, 320534846571637606, 100697469688841817], [13248845538017813727, 14468596293127334607, 8656550560197473959, 2161239339877761890], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12362648763242643187, 13940026059969050459, 6027715406760125980, 2781413989188023337]]], 'proof': '1ece89bb01f80c0829026081ae5558d4d6fbb7347311be4b21b8b89e7e42bfb7078a69191dbfc31e1873d596b492a5ae9a599349e8cba18db69ff91a76dc59a41527fb20a6b65bc4ea7687e3a2df3307eb6c3134121c7ff624c92292d86171751d7496d8be004190b11a393f4ad05c3d2de6d1aae53d5210dbc16048f350f9cf2ce86cc113c462e83d2b0608a9b88aa28a6b5de96ef0c151bec1cbf0618405770471913d8efd53d6212bb1ffce224221b343a0a81b1f0eee91905e71a6dbf1210f62d553b5d61772c4c80eb5a19bcaf6b5337248975fffe83598fcf08ae432b3272188c85b274937b54c77f81124b28c9e7024e7deb1d8ee3bfd3bbceef4f00028195ff5ea5711e0871410610bc3f53b0ca2397958aeb119b1169c7d6096422e0aa17a4859ad681733e6bebbbb023c99d53b270e

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

12.75